In [5]:
import pandas as pd
import numpy as np

In [8]:
pd.read_csv('data/2002年-2018年上海机动车拍照拍卖.csv').head()

,Date,Total number of license issued,lowest price,avg price,Total number of applicants
0,2-Jan,1400,13600,14735,3718
1,2-Feb,1800,13100,14057,4590
2,2-Mar,2000,14300,14662,5190
3,2-Apr,2300,16000,16334,4806
4,2-May,2350,17800,18357,4665


# Pandas 教程（上）综合练习

## 一、2002 年-2018 年上海机动车拍照拍卖

1. 哪一次拍卖的中标率首次小于 5%？
2. 按年统计拍卖最低价的下列统计量：最大值、均值、0.75 分位数，要求
显示在同一张表上。
3. 将第一列时间列拆分成两个列，一列为年份（格式为 20××），另一列为
月份（英语缩写），添加到列表作为第一第二列，并将原表第一列删除，
其他列依次向后顺延。
4. 现在将表格行索引设为多级索引，外层为年份，内层为原表格第二至第
五列的变量名，列索引为月份。
5. 一般而言某个月最低价与上月最低价的差额，会与该月均值与上月均值
的差额具有相同的正负号，哪些拍卖时间不具有这个特点？
6. 将某一个月牌照发行量与其前两个月发行量均值的差额定义为发行增
益，最初的两个月用 0 填充，求发行增益极值出现的时间

In [11]:
data = pd.read_csv('data/2002年-2018年上海机动车拍照拍卖.csv')

In [13]:
data1 = data.assign(probability = data['Total number of license issued'] / data['Total number of applicants'])
data1.head()

,Date,Total number of license issued,lowest price,avg price,Total number of applicants,probability
0,2-Jan,1400,13600,14735,3718,0.376547
1,2-Feb,1800,13100,14057,4590,0.392157
2,2-Mar,2000,14300,14662,5190,0.385356
3,2-Apr,2300,16000,16334,4806,0.478568
4,2-May,2350,17800,18357,4665,0.503751


In [14]:
data1.sort_values(by = 'probability').head(1)

,Date,Total number of license issued,lowest price,avg price,Total number of applicants,probability
169,16-Mar,8310,83100,83148,221109,0.037583


In [17]:
data1[data1['probability']<0.05].head(1)

,Date,Total number of license issued,lowest price,avg price,Total number of applicants,probability
159,15-May,7482,79000,79099,156007,0.047959


In [19]:
data2 = data1.assign(year = data1['Date'] . apply(lambda x:x.split('-')[0]))
data2[data2['probability']<0.05].sort_values(by = 'probability').head(1)

,Date,Total number of license issued,lowest price,avg price,Total number of applicants,probability,year
169,16-Mar,8310,83100,83148,221109,0.037583,16


In [30]:
data3 = data2.groupby('year')
new_dataframe = pd.DataFrame()
new_dataframe['max'] = data3['lowest price'].max()
new_dataframe['mean'] = data3['lowest price'].mean()
new_dataframe['quantile'] = data2['lowest price'].quantile(0.75)
new_dataframe

KeyError: 'Column not found: lowest price'

In [31]:
Year = data['Date'].apply(lambda x: str(20) + ("%02d" % int(x.split('-')[0])))
Month = data['Date'].apply(lambda x: x.split('-')[1])

data4 = data.drop(columns='Date')
data4.insert(0, 'Year', Year)
data4.insert(1, 'Month', Month)
data4.head()

,Year,Month,Total number of license issued,lowest price,avg price,Total number of applicants
0,2002,Jan,1400,13600,14735,3718
1,2002,Feb,1800,13100,14057,4590
2,2002,Mar,2000,14300,14662,5190
3,2002,Apr,2300,16000,16334,4806
4,2002,May,2350,17800,18357,4665


In [34]:
data5 = data4.set_index(['Year', 'Total number of license issued', 'lowest price', 'avg price', 'Total number of applicants'])
data5.head()

KeyError: "None of ['lowest price'] are in the columns"

In [35]:
da = data[['Date','lowest price','avg price']]

for index in range(1, len(da)):
    if ((da.iloc[index]['lowest price'] - da.iloc[index-1]['lowest price'])\
    * (da.iloc[index]['avg price'] - da.iloc[index-1]['avg price'])) < 0:
        print(da.iloc[index]['Date'])

KeyError: "['lowest price'] not in index"

In [36]:
dat = data[['Date','Total number of license issued']]
gain = [0,0]
string = 'Total number of license issued'
for index in range(2, len(dat)):
    gain.append(data.iloc[index][string] - np.mean([data.iloc[index-1][string], data.iloc[index-2][string]]))
dat['gain'] = gain

print('增益极大值出现时间：', dat[dat['gain'] == dat['gain'].max()]['Date'].values)
print('增益极小值出现时间：', dat[dat['gain'] == dat['gain'].min()]['Date'].values)

增益极大值出现时间： ['8-Jan']
增益极小值出现时间： ['8-Apr']


F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# 二、2007 年-2019 年俄罗斯机场货运航班运载量

1. 求每年货运航班总运量。
2. 每年记录的机场都是相同的吗？
3. 按年计算 2010 年-2015 年全年货运量记录为 0 的机场航班比例。
4. 若某机场至少存在 5 年或以上满足所有月运量记录都为 0，则将其所有
年份的记录信息从表中删除，并返回处理后的表格
5. 采用一种合理的方式将所有机场划分为东南西北四个分区，并给出 2017
年-2019 年货运总量最大的区域。
6. 在统计学中常常用秩代表排名，现在规定某个机场某年某个月的秩为该
机场该月在当年所有月份中货运量的排名（例如 *** 机场 19 年 1 月运
量在整个 19 年 12 个月中排名第一，则秩为 1），那么判断某月运量情
况的相对大小的秩方法为将所有机场在该月的秩排名相加，并将这个量
定义为每一个月的秩综合指数，请根据上述定义计算 2016 年 12 个月
的秩综合指数。

In [37]:
data = pa.read_csv('data/2007年-2019年俄罗斯货运航班运载量.csv')
data.head()

,Airport name,Year,January,February,March,April,May,June,July,August,September,October,November,December,Whole year,Airport coordinates
0,Abakan,2019,44.70,66.21,72.7,75.82,100.34,78.38,63.88,73.06,66.74,75.44,110.5,89.8,917.57,"(Decimal('91.399735'), Decimal('53.751351'))"
1,Aikhal,2019,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,"(Decimal('111.543324'), Decimal('65.957161'))"
2,Loss,2019,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,"(Decimal('125.398355'), Decimal('58.602489'))"
3,Amderma,2019,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,"(Decimal('61.577429'), Decimal('69.759076'))"
4,Anadyr (Carbon),2019,81.63,143.01,260.9,304.36,122.00,106.87,84.99,130.00,102.00,118.00,94.0,199.0,1746.76,"(Decimal('177.738273'), Decimal('64.713433'))"


In [39]:
data1 = data.groupby('Year')
data1['Whole year'].sum()

Year
2007    659438.23
2008    664682.46
2009    560809.77
2010    693033.98
2011    818691.71
2012    846388.03
2013    792337.08
2014    729457.12
2015    630208.97
2016    679370.15
2017    773662.28
2018    767095.28
2019    764606.27
Name: Whole year, dtype: float64

In [40]:
display(data1['Airport name'].count())

Year
2007    292
2008    292
2009    292
2010    292
2011    292
2012    292
2013    292
2014    292
2015    292
2016    292
2017    292
2018    248
2019    251
Name: Airport name, dtype: int64

In [41]:
data2 = data.groupby('Airport name')['Year']
display(data2.count())



Airport name
Abakan           13
Achinsk          11
Aikhal           13
Amderma          13
Anadyr           12
                 ..
Лешуконское      13
Мотыгино         13
Нюрба            13
Среднеколымск    13
Таксимо          13
Name: Year, Length: 297, dtype: int64

In [43]:
print( '有 %d个机场并不是每年都被统计。' %  len(data2.count()[(data2.count())<(2019-2007+1)]))

有 63个机场并不是每年都被统计。


In [44]:
data3 = data.set_index(['Year'])
data4 = data3.sort_index().loc[2010:2015].groupby('Year')
display(data4['Whole year'].agg( lambda x: print( '年份：', x.index[0], '\t 比例：%.2f' % (len(x[x==0]) / len(x) * 100), '%' ) ))

年份： 2010 	 比例：76.71 %
年份： 2011 	 比例：77.05 %
年份： 2012 	 比例：77.05 %
年份： 2013 	 比例：77.05 %
年份： 2014 	 比例：77.05 %
年份： 2015 	 比例：77.05 %


Year
2010   NaN
2011   NaN
2012   NaN
2013   NaN
2014   NaN
2015   NaN
Name: Whole year, dtype: float64

In [45]:
data5 = data.set_index('Airport name')
d6 = data5.groupby('Airport name')
zero_airport = pd.DataFrame(d6['Whole year'].apply( lambda x: len(x[x==0])>5 ))
data5.drop(zero_airport[zero_airport['Whole year']==True].index).head()


,Year,January,February,March,April,May,June,July,August,September,October,November,December,Whole year,Airport coordinates
Airport name,,,,,,,,,,,,,,,
Abakan,2019,44.70,66.21,72.70,75.82,100.34,78.38,63.88,73.06,66.74,75.44,110.50,89.80,917.57,"(Decimal('91.399735'), Decimal('53.751351'))"
Anadyr (Carbon),2019,81.63,143.01,260.90,304.36,122.00,106.87,84.99,130.00,102.00,118.00,94.00,199.00,1746.76,"(Decimal('177.738273'), Decimal('64.713433'))"
Anapa (Vitjazevo),2019,45.92,53.15,54.00,54.72,52.00,67.45,172.31,72.57,70.00,63.00,69.00,82.10,856.22,"(Decimal('37.341511'), Decimal('45.003748'))"
Arkhangelsk (Talagy),2019,85.61,118.70,131.39,144.82,137.95,140.18,128.56,135.68,124.75,139.60,210.27,307.10,1804.61,"(Decimal('40.714892'), Decimal('64.596138'))"
Astrakhan (Narimanovo),2019,51.75,61.08,65.60,71.84,71.38,63.95,164.86,79.46,85.21,87.23,79.06,99.16,980.58,"(Decimal('47.999896'), Decimal('46.287344'))"


In [47]:
import re
dd = data.set_index(['Year']).sort_index().loc[2017:2019].loc[:, ['Whole year','Airport coordinates']]
# 坐标 提取
longitude = []
latitude = []
for i in range(0, len(dd['Airport coordinates'])):
    string = re.findall(r"\d+\.?\d*", str(dd['Airport coordinates'].iloc[i]))
    
    if string==[]:
        longitude.append(np.nan)
        latitude.append(np.nan)
    else:
        longitude.append(float(string[0]))
        latitude.append( float(string[1]))

dd['Longitude'] = longitude
dd['Latitude'] = latitude
dd = dd.dropna()   # 删掉 Not found 的机场
cuts_long = pd.cut(dd['Longitude'],bins=[0, dd['Longitude'].median(), 180])
dd['cuts_long'] = cuts_long
cuts_la = pd.cut(dd['Latitude'],bins=[0, dd['Latitude'].median(), 90])
dd['cuts_la'] = cuts_la
display(dd.groupby(['cuts_long', 'cuts_la']).count())
asa = dd.groupby(['cuts_long', 'cuts_la'])
print( '2017 年 - 2019 年货运总量最大区域的 经度范围：%s，维度范围：%s' \
      % (asa['Whole year'].sum().idxmax()[0], asa['Whole year'].sum().idxmax()[1]) )


Whole year  Airport coordinates  Longitude  \
cuts_long       cuts_la                                                      
(0.0, 82.049]   (0.0, 58.498]          187                  187        187   
                (58.498, 90.0]         196                  196        196   
(82.049, 180.0] (0.0, 58.498]          195                  195        195   
                (58.498, 90.0]         185                  185        185   

                                Latitude  
cuts_long       cuts_la                   
(0.0, 82.049]   (0.0, 58.498]        187  
                (58.498, 90.0]       196  
(82.049, 180.0] (0.0, 58.498]        195  
                (58.498, 90.0]       185

2017 年 - 2019 年货运总量最大区域的 经度范围：(0.0, 82.049]，维度范围：(0.0, 58.498]


In [48]:
d1 = data[data['Year']==2016]

month = d1.columns[2:14]
Rank = pd.DataFrame(index=d1.index, columns=month)
for index in d1.index:
    aa = d1.loc[index,month].sort_values(ascending=False)
    rank = 1
    for mon in aa.index:
        Rank.loc[index, mon] = rank
        rank = rank + 1

print(Rank.apply(lambda x: x.sum()))


January      3406
February     3076
March        2730
April        2432
May          2276
June         2047
July         1854
August       1527
September    1269
October      1009
November      728
December      422
dtype: int64


In [49]:
d2 = pd.read_csv('data/美国确证数.csv')
d3 = pd.read_csv('data/美国死亡数.csv')
display(d2.head())
display(d3.head())


,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2020/4/17,2020/4/18,2020/4/19,2020/4/20,2020/4/21,2020/4/22,2020/4/23,2020/4/24,2020/4/25,2020/4/26
0,84001001,US,USA,840,1001,Autauga,Alabama,US,32.539527,-86.644082,...,26,25,26,28,30,32,33,36,36,37
1,84001003,US,USA,840,1003,Baldwin,Alabama,US,30.727750,-87.722071,...,103,109,112,117,123,132,143,147,147,161
2,84001005,US,USA,840,1005,Barbour,Alabama,US,31.868263,-85.387129,...,15,18,20,22,28,29,30,32,32,33
3,84001007,US,USA,840,1007,Bibb,Alabama,US,32.996421,-87.125115,...,24,26,28,32,32,34,33,34,34,38
4,84001009,US,USA,840,1009,Blount,Alabama,US,33.982109,-86.567906,...,20,20,21,22,26,29,31,31,31,34


,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2020/4/17,2020/4/18,2020/4/19,2020/4/20,2020/4/21,2020/4/22,2020/4/23,2020/4/24,2020/4/25,2020/4/26
0,84001001,US,USA,840,1001,Autauga,Alabama,US,32.539527,-86.644082,...,2,2,2,1,1,2,2,2,2,2
1,84001003,US,USA,840,1003,Baldwin,Alabama,US,30.727750,-87.722071,...,2,2,2,3,3,3,3,3,3,3
2,84001005,US,USA,840,1005,Barbour,Alabama,US,31.868263,-85.387129,...,0,0,0,0,0,0,0,0,0,0
3,84001007,US,USA,840,1007,Bibb,Alabama,US,32.996421,-87.125115,...,0,0,0,0,0,0,0,0,0,0
4,84001009,US,USA,840,1009,Blount,Alabama,US,33.982109,-86.567906,...,0,0,0,0,0,0,0,0,0,0


In [50]:
d3[['Population', '2020/4/26']].corr()


,Population,2020/4/26
Population,1.000000,0.403844
2020/4/26,0.403844,1.000000


In [51]:
# 时序数据，是累计的，因此只取2020/4/1日
d4 = d3[['Admin2', 'Province_State', '2020/4/1']]
d5 = d4.groupby('Province_State')
# display(groupby_Q3_2.first())

d5['2020/4/1'].apply(lambda x: len(x[x==0]) / len(x) )


Province_State
Alabama                 0.776119
Alaska                  0.965517
Arizona                 0.800000
Arkansas                0.946667
California              0.568966
Colorado                0.734375
Connecticut             0.125000
Delaware                0.000000
District of Columbia    0.000000
Florida                 0.641791
Georgia                 0.748428
Hawaii                  1.000000
Idaho                   0.886364
Illinois                0.862745
Indiana                 0.728261
Iowa                    0.939394
Kansas                  0.952381
Kentucky                0.941667
Louisiana               0.500000
Maine                   0.812500
Maryland                0.583333
Massachusetts           0.142857
Michigan                0.734940
Minnesota               0.965517
Mississippi             0.768293
Missouri                0.913043
Montana                 0.946429
Nebraska                0.956989
Nevada                  0.882353
New Hampshire           0.80